In [1]:
# 从sklearn 调入所需要的包
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score,roc_auc_score

In [2]:
path='D:/dataset/purchse_predict/'
train = pd.read_csv(path+'input/train_set.csv')
test = pd.read_csv(path+'input/test_set.csv')
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         25317 non-null  int64 
 1   age        25317 non-null  int64 
 2   job        25317 non-null  object
 3   marital    25317 non-null  object
 4   education  25317 non-null  object
 5   default    25317 non-null  object
 6   balance    25317 non-null  int64 
 7   housing    25317 non-null  object
 8   loan       25317 non-null  object
 9   contact    25317 non-null  object
 10  day        25317 non-null  int64 
 11  month      25317 non-null  object
 12  duration   25317 non-null  int64 
 13  campaign   25317 non-null  int64 
 14  pdays      25317 non-null  int64 
 15  previous   25317 non-null  int64 
 16  poutcome   25317 non-null  object
 17  y          25317 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 3.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 

In [3]:
category_feats_0 = ['job','marital','default','education','housing','contact','loan','month','poutcome']

In [4]:
train.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,12659.000000,40.935379,1357.555082,15.835289,257.732393,2.772050,40.248766,0.591737,0.116957
std,7308.532719,10.634289,2999.822811,8.319480,256.975151,3.136097,100.213541,2.568313,0.321375
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6330.000000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,12659.000000,39.000000,448.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,18988.000000,48.000000,1435.000000,21.000000,317.000000,3.000000,-1.000000,0.000000,0.000000
max,25317.000000,95.000000,102127.000000,31.000000,3881.000000,55.000000,854.000000,275.000000,1.000000


In [5]:
test.describe()

,ID,age,balance,day,duration,campaign,pdays,previous
count,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000,10852.000000
mean,30743.500000,41.040638,1393.157298,15.680151,257.206137,2.770365,39.954755,0.546443
std,3132.846895,10.652369,3318.497101,8.302317,250.480906,3.063481,99.524056,1.805938
min,25318.000000,18.000000,-2604.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,28030.750000,33.000000,72.000000,8.000000,102.000000,1.000000,-1.000000,0.000000
50%,30743.500000,39.000000,450.000000,16.000000,181.000000,2.000000,-1.000000,0.000000
75%,33456.250000,49.000000,1440.250000,21.000000,322.000000,3.000000,-1.000000,0.000000
max,36169.000000,94.000000,81204.000000,31.000000,3102.000000,58.000000,871.000000,51.000000


In [6]:
train.corr()

,ID,age,balance,day,duration,campaign,pdays,previous,y
ID,1.000000,0.008465,0.032719,-0.020171,0.229149,-0.038265,0.065807,0.047028,0.556627
age,0.008465,1.000000,0.093740,-0.016070,0.000416,0.006171,-0.026431,0.006575,0.029916
balance,0.032719,0.093740,1.000000,0.010245,0.026042,-0.010419,0.001032,0.015792,0.057564
day,-0.020171,-0.016070,0.010245,1.000000,-0.031946,0.168830,-0.092892,-0.050706,-0.031886
duration,0.229149,0.000416,0.026042,-0.031946,1.000000,-0.087780,0.000040,0.001315,0.394746
campaign,-0.038265,0.006171,-0.010419,0.168830,-0.087780,1.000000,-0.089224,-0.031667,-0.075173
pdays,0.065807,-0.026431,0.001032,-0.092892,0.000040,-0.089224,1.000000,0.411688,0.107565
previous,0.047028,0.006575,0.015792,-0.050706,0.001315,-0.031667,0.411688,1.000000,0.088337
y,0.556627,0.029916,0.057564,-0.031886,0.394746,-0.075173,0.107565,0.088337,1.000000


In [7]:
train.job.value_counts()

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

In [8]:
# data=pd.concat([train,test])

In [9]:
test['y']=-1
print(len(test.columns))

18


In [10]:
data = train.append(test).reset_index(drop=True)
# data = pd.concat([train,test]).reset_index(drop=True)
print(data.head())

   ID  age         job   marital  education default  balance housing loan  \
0   1   43  management   married   tertiary      no      291     yes   no   
1   2   42  technician  divorced    primary      no     5076     yes   no   
2   3   47      admin.   married  secondary      no      104     yes  yes   
3   4   28  management    single  secondary      no     -994     yes  yes   
4   5   42  technician  divorced  secondary      no     2974     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0   unknown    9   may       150         2     -1         0  unknown  0  
1  cellular    7   apr        99         1    251         2    other  0  
2  cellular   14   jul        77         2     -1         0  unknown  0  
3  cellular   18   jul       174         2     -1         0  unknown  0  
4   unknown   21   may       187         5     -1         0  unknown  0  


In [11]:
path='./'
import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
    #warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 

In [12]:
cate_features = ['job', 'marital','education','default','housing','loan','contact','month','poutcome']
num_features = ['age', 'balance','day','duration','campaign','pdays','previous']
feature1 = cate_features + num_features

In [13]:
def feature_count(data, features):
    feature_name = 'count'
    for i in features:
        feature_name += '_' + i
    temp = data.groupby(features).size().reset_index().rename(columns={0: feature_name})
    data = data.merge(temp, 'left', on=features)
    return data,feature_name
ll=[]
for f in['campaign', 'contact','default','education','housing','job','loan','marital','poutcome','pdays','previous']:
    data,_=feature_count(data,['day','month',f])
    ll.append(_)

In [14]:
ll

['count_day_month_campaign',
 'count_day_month_contact',
 'count_day_month_default',
 'count_day_month_education',
 'count_day_month_housing',
 'count_day_month_job',
 'count_day_month_loan',
 'count_day_month_marital',
 'count_day_month_poutcome',
 'count_day_month_pdays',
 'count_day_month_previous']

In [15]:
sparse_feature= ['campaign','contact','default','education','housing','job','loan','marital','month','poutcome']
dense_feature=['age', 'balance','day','duration','campaign','pdays','previous']
crossf=[]
for d in tqdm(sparse_feature):
    aggs={}
    for s in sparse_feature:
        aggs[s]=['count','nunique']
        crossf+=['count_'+str(d)+'_'+str(s),'nunique_'+str(d)+'_'+str(s)]
        temp=data.groupby(d)[s].agg([('count_'+str(d)+'_'+str(s),'count'),('nunique_'+str(d)+'_'+str(s),'nunique')]).reset_index()
        data=pd.merge(data,temp,on=d,how='left')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:18<00:00,  1.84s/it]


In [16]:
temp.head()

,poutcome,count_poutcome_poutcome,nunique_poutcome_poutcome
0,failure,3938,1
1,other,1475,1
2,success,1202,1
3,unknown,29554,1


In [17]:
for d in tqdm(sparse_feature):
    aggs={}
    for s in dense_feature:
#         aggs[s]=['count','nunique']
        crossf+=[str(d)+'_mean_'+str(s),str(d)+'_max_'+str(s),str(d)+'_min_'+str(s),str(d)+'_std_'+str(s)]
        temp = data.groupby(d)[s].agg([(str(d)+'_mean_'+str(s),'mean'),(str(d)+'_max_'+str(s),'max'),(str(d)+'_min_'+str(s),'min'),(str(d)+'_std_'+str(s),'std')]).reset_index()
        data=pd.merge(data,temp,on=d,how='left')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [03:05<00:00, 18.51s/it]


In [18]:
temp.head()

,poutcome,poutcome_mean_previous,poutcome_max_previous,poutcome_min_previous,poutcome_std_previous
0,failure,2.874556,51,1,3.036148
1,other,4.019661,275,1,8.374317
2,success,3.032446,22,1,2.467733
3,unknown,0.000575,7,0,0.052992


In [19]:
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    data[i] = lbl.fit_transform(data[i].astype(str))
    
# for i in cat_col:
#     data[i] = data[i].astype('category')

  0%|          | 0/9 [00:00<?, ?it/s]

In [20]:
for col in category_feats_0 :
    onehot_feats = pd.get_dummies(data[col], prefix = col)
    data.drop([col], axis = 1, inplace = True)
    data = pd.concat([data, onehot_feats], axis = 1)

In [21]:
data.shape

(36169, 544)

In [22]:
data = data.fillna(-1)

In [23]:
continuous_feature = [i for i in data.columns if i not in ['ID','y']]

In [24]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
scaler = MinMaxScaler()
for col in continuous_feature:
    data[col] = scaler.fit_transform(data[col].values.reshape(-1, 1))

In [25]:
data.head()

,ID,age,balance,day,duration,campaign,pdays,previous,y,count_day_month_campaign,...,month_6,month_7,month_8,month_9,month_10,month_11,poutcome_0,poutcome_1,poutcome_2,poutcome_3
0,1,0.324675,0.075445,0.266667,0.038650,0.017544,0.000000,0.000000,0,0.293651,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,0.311688,0.118888,0.200000,0.025509,0.000000,0.288991,0.007273,0,0.105820,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,3,0.376623,0.073748,0.433333,0.019840,0.017544,0.000000,0.000000,0,0.203704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,0.129870,0.063779,0.566667,0.044834,0.017544,0.000000,0.000000,0,0.126984,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,0.311688,0.099804,0.666667,0.048183,0.070175,0.000000,0.000000,0,0.037037,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [26]:
import numpy as np
import pandas as pd


In [27]:
train=data[data['y']!=-1]
test=data[data['y']==-1]

In [28]:
X_train = train[continuous_feature]
y_train = train['y']

In [29]:
X_test = test[continuous_feature]
y_test = test['y']

In [30]:
train_x, vali_x, train_y, vali_y = train_test_split(X_train , y_train, test_size=0.3, random_state=42)

In [31]:

clf = lgb.LGBMRegressor(objective='binary',
                       subsample= 0.8,
                       min_child_weight= 0.5,
                       colsample_bytree= 0.7,
                       num_leaves =100,
                       max_depth = 12,
                       learning_rate = 0.05,
                       n_estimators = 10,
                       boosting_type='gbdt'
                       )
clf.fit(train_x,train_y,
        eval_set = [(train_x,train_y),(vali_x,vali_y)],
       eval_names = ['train','val'],
       eval_metric = 'binary_logloss'
       )
model = clf.booster_

[1]	train's binary_logloss: 0.33811	val's binary_logloss: 0.340186
[2]	train's binary_logloss: 0.31982	val's binary_logloss: 0.325223
[3]	train's binary_logloss: 0.304312	val's binary_logloss: 0.31281
[4]	train's binary_logloss: 0.291382	val's binary_logloss: 0.30195
[5]	train's binary_logloss: 0.279951	val's binary_logloss: 0.292493
[6]	train's binary_logloss: 0.270041	val's binary_logloss: 0.284575
[7]	train's binary_logloss: 0.264168	val's binary_logloss: 0.280626
[8]	train's binary_logloss: 0.255364	val's binary_logloss: 0.273953
[9]	train's binary_logloss: 0.247463	val's binary_logloss: 0.267447
[10]	train's binary_logloss: 0.240324	val's binary_logloss: 0.261561


In [33]:
gbdt_feats_train = model.predict(train_x, pred_leaf= True)
gbdt_feats_test = model.predict(vali_x, pred_leaf= True)
gbdt_feats_name = ['gbdt_leaf_' + str(i) for i in range(gbdt_feats_train.shape[1])]
df_train_gbdt_feats = pd.DataFrame(gbdt_feats_train, columns=gbdt_feats_name)
df_vali_gbdt_feats = pd.DataFrame(gbdt_feats_test, columns=gbdt_feats_name)

In [35]:
gbdt_feats_train[0]

array([75, 59, 14, 40, 52, 38, 54, 17, 27, 27])

In [38]:
gbdt_feats_name = ['gbdt_leaf_'+str(i) for i in range(gbdt_feats_train.shape[1])]
df_train_gbdt_feats = pd.DataFrame(gbdt_feats_train,columns = gbdt_feats_name)
df_vali_gbdt_feats = pd.DataFrame(gbdt_feats_test, columns = gbdt_feats_name)

In [39]:
gbdt_feats_train.shape

(17721, 10)

In [40]:
df_train_gbdt_feats.head()

,gbdt_leaf_0,gbdt_leaf_1,gbdt_leaf_2,gbdt_leaf_3,gbdt_leaf_4,gbdt_leaf_5,gbdt_leaf_6,gbdt_leaf_7,gbdt_leaf_8,gbdt_leaf_9
0,75,59,14,40,52,38,54,17,27,27
1,90,14,4,55,57,94,34,87,12,46
2,30,11,9,89,5,70,1,7,37,74
3,30,11,9,7,5,9,65,7,37,7
4,30,11,9,7,5,9,34,7,37,7


In [41]:
gbdt_feats_name

['gbdt_leaf_0',
 'gbdt_leaf_1',
 'gbdt_leaf_2',
 'gbdt_leaf_3',
 'gbdt_leaf_4',
 'gbdt_leaf_5',
 'gbdt_leaf_6',
 'gbdt_leaf_7',
 'gbdt_leaf_8',
 'gbdt_leaf_9']

In [42]:
train_x = train_x.reset_index(drop=True)
vali_x = vali_x.reset_index(drop=True)


In [43]:
train_x.head()

,age,balance,day,duration,campaign,pdays,previous,count_day_month_campaign,count_day_month_contact,count_day_month_default,...,month_6,month_7,month_8,month_9,month_10,month_11,poutcome_0,poutcome_1,poutcome_2,poutcome_3
0,0.311688,0.074919,0.233333,0.025251,0.000000,0.266055,0.014545,0.121693,0.133556,0.096956,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.467532,0.087457,0.533333,0.137593,0.052632,0.000000,0.000000,0.058201,0.445743,0.311161,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.428571,0.063343,0.866667,0.070085,0.017544,0.000000,0.000000,0.373016,0.636060,0.497182,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.129870,0.082382,0.900000,0.050502,0.000000,0.000000,0.000000,0.513228,0.485810,0.381060,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.103896,0.078124,0.366667,0.055398,0.000000,0.000000,0.000000,0.571429,0.370618,0.700113,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [44]:
train_label = pd.concat([train_x,df_train_gbdt_feats],axis=1)
vali_label = pd.concat([vali_x,df_vali_gbdt_feats],axis=1)

In [45]:
train_label.head()

,age,balance,day,duration,campaign,pdays,previous,count_day_month_campaign,count_day_month_contact,count_day_month_default,...,gbdt_leaf_0,gbdt_leaf_1,gbdt_leaf_2,gbdt_leaf_3,gbdt_leaf_4,gbdt_leaf_5,gbdt_leaf_6,gbdt_leaf_7,gbdt_leaf_8,gbdt_leaf_9
0,0.311688,0.074919,0.233333,0.025251,0.000000,0.266055,0.014545,0.121693,0.133556,0.096956,...,75,59,14,40,52,38,54,17,27,27
1,0.467532,0.087457,0.533333,0.137593,0.052632,0.000000,0.000000,0.058201,0.445743,0.311161,...,90,14,4,55,57,94,34,87,12,46
2,0.428571,0.063343,0.866667,0.070085,0.017544,0.000000,0.000000,0.373016,0.636060,0.497182,...,30,11,9,89,5,70,1,7,37,74
3,0.129870,0.082382,0.900000,0.050502,0.000000,0.000000,0.000000,0.513228,0.485810,0.381060,...,30,11,9,7,5,9,65,7,37,7
4,0.103896,0.078124,0.366667,0.055398,0.000000,0.000000,0.000000,0.571429,0.370618,0.700113,...,30,11,9,7,5,9,34,7,37,7


In [46]:
vali_label.shape

(7596, 552)

In [47]:
train_label.shape

(17721, 552)

In [49]:
vali_x.shape


(7596, 542)

In [50]:
train_len = train_label.shape[0]
data = pd.concat([train_label,vali_label])
data = data.fillna(0)

In [51]:
for col in gbdt_feats_name:
    # print('this is feature:',col)
    onehot_feats = pd.get_dummies(data[col], prefix=col)
    data.drop([col], axis=1,inplace= True)
    data = pd.concat([data, onehot_feats],axis=1)

In [52]:
train_x = data[:train_len]
vali_x = data[train_len:]

In [53]:
train_x.shape

(17721, 1542)

In [54]:
train_y.shape

(17721,)

In [56]:
vali_x.shape

(7596, 1542)

In [57]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
lr = LogisticRegression()
lr.fit(train_x,train_y)
tr_logloss = log_loss(train_y, lr.predict_proba(train_x)[:,1])
tr_logloss

0.1089384472374046

In [58]:
res = lr.predict_proba(vali_x)[:,1]
res.shape

(7596,)

In [59]:
print('线下AUC:' +str(roc_auc_score(vali_y,res)))

线下AUC:0.9179734479745872
